DDL vs DML vs DQL
DDL (Data Definition Language): Defines and changes database structure (tables, schemas, indexes). Example: CREATE TABLE Customers (CustomerID INT PRIMARY KEY, CustomerName VARCHAR(100));

DML (Data Manipulation Language): Works with the data inside tables (insert, update, delete). Example: INSERT INTO Customers (CustomerID, CustomerName) VALUES (1, 'Alice Wonderland');

DQL (Data Query Language): Retrieves data from tables, typically using SELECT. Example: SELECT CustomerName FROM Customers WHERE CustomerID = 1;
​

Q2: Purpose of constraints (3 types)
Purpose: SQL constraints enforce rules on table data to maintain accuracy and integrity (no invalid, duplicate, or inconsistent data).

PRIMARY KEY: Uniquely identifies each row and cannot be NULL.

Scenario: CustomerID in Customers table so each customer is uniquely tracked.

FOREIGN KEY: Enforces valid relationships between tables.

Scenario: CustomerID in Orders must exist in Customers to avoid orphan orders.

UNIQUE: Ensures all values in a column are different.

Scenario: Email in Customers so no two customers share the same email.
​

(Other acceptable constraints: NOT NULL, CHECK, DEFAULT.)

Q3: LIMIT vs OFFSET, third page with 10 rows
LIMIT: Specifies how many rows to return, e.g. LIMIT 10 returns 10 rows.

OFFSET: Skips a given number of rows before starting to return rows, e.g. OFFSET 20 skips 20 rows.

For page 3 with 10 records per page:

Rows per page: 10

Page 3 starts after first 20 rows → OFFSET 20

SELECT *
FROM some_table
ORDER BY some_column
LIMIT 10
OFFSET 20;


Q4: What is a CTE + example
A Common Table Expression (CTE) is a named temporary result set defined with WITH that exists only for a single statement. It improves readability, allows reuse of subqueries, and helps with hierarchical or recursive queries.


In [ ]:
WITH HighValueOrders AS (
    SELECT CustomerID, OrderID, TotalAmount
    FROM Orders
    WHERE TotalAmount > 500
)
SELECT c.CustomerName, h.OrderID, h.TotalAmount
FROM HighValueOrders h
JOIN Customers c ON c.CustomerID = h.CustomerID;


5: Normalization and 1NF, 2NF, 3NF
SQL normalization organizes table structures to reduce redundancy and avoid update anomalies while ensuring data integrity.
​

1NF (First Normal Form):

Each column holds atomic (indivisible) values.

No repeating groups or arrays; each row-column intersection has a single value.

2NF (Second Normal Form):

Table is in 1NF.

Every non-key column is fully functionally dependent on the entire primary key (no partial dependency on part of a composite key).

3NF (Third Normal Form):

Table is in 2NF.

No transitive dependencies: non-key columns depend only on the key, not on other non-key columns.
​

In [ ]:
Q6: ECommerceDB – create tables + inserts
Create database and tables
sql
CREATE DATABASE ECommerceDB;
USE ECommerceDB;

CREATE TABLE Categories (
    CategoryID INT PRIMARY KEY,
    CategoryName VARCHAR(50) NOT NULL UNIQUE
);

CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(100) NOT NULL UNIQUE,
    CategoryID INT,
    Price DECIMAL(10,2) NOT NULL,
    StockQuantity INT,
    CONSTRAINT fk_products_category
        FOREIGN KEY (CategoryID)
        REFERENCES Categories(CategoryID)
);

CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(100) NOT NULL,
    Email VARCHAR(100) UNIQUE,
    JoinDate DATE
);

CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATE NOT NULL,
    TotalAmount DECIMAL(10,2),
    CONSTRAINT fk_orders_customer
        FOREIGN KEY (CustomerID)
        REFERENCES Customers(CustomerID)
);INSERT INTO Categories (CategoryID, CategoryName) VALUES
(1, 'Electronics'),
(2, 'Books'),
(3, 'Home Goods'),
(4, 'Apparel');

INSERT INTO Products (ProductID, ProductName, CategoryID, Price, StockQuantity) VALUES
(101, 'Laptop Pro', 1, 1200.00, 50),
(102, 'SQL Handbook', 2, 45.50, 200),
(103, 'Smart Speaker', 1, 99.99, 150),
(104, 'Coffee Maker', 3, 75.00, 80),
(105, 'Novel : The Great SQL', 2, 25.00, 120),
(106, 'Wireless Earbuds', 1, 150.00, 100),
(107, 'Blender X', 3, 120.00, 60),
(108, 'T-Shirt Casual', 4, 20.00, 300);

INSERT INTO Customers (CustomerID, CustomerName, Email, JoinDate) VALUES
(1, 'Alice Wonderland', 'alice@example.com', '2023-01-10'),
(2, 'Bob the Builder', 'bob@example.com', '2022-11-25'),
(3, 'Charlie Chaplin', 'charlie@example.com', '2023-03-01'),
(4, 'Diana Prince', 'diana@example.com', '2021-04-26');

INSERT INTO Orders (OrderID, CustomerID, OrderDate, TotalAmount) VALUES
(1001, 1, '2023-04-26', 1245.50),
(1002, 2, '2023-10-12', 99.99),
(1003, 1, '2023-07-01', 145.00),
(1004, 3, '2023-01-14', 150.00),
(1005, 2, '2023-09-24', 120.00),
(1006, 1, '2023-06-19', 20.00);


Q7: CustomerName, Email, TotalNumberofOrders (including 0)
Use a LEFT JOIN from Customers to Orders and aggregate:

In [ ]:
SELECT
    c.CustomerName,
    c.Email,
    COALESCE(COUNT(o.OrderID), 0) AS TotalNumberofOrders
FROM Customers c
LEFT JOIN Orders o
    ON c.CustomerID = o.CustomerID
GROUP BY
    c.CustomerID,
    c.CustomerName,
    c.Email
ORDER BY
    c.CustomerName;


Q8: Product info with Category

In [ ]:
SELECT
    p.ProductName,
    p.Price,
    p.StockQuantity,
    c.CategoryName
FROM Products p
JOIN Categories c
    ON p.CategoryID = c.CategoryID
ORDER BY
    c.CategoryName,
    p.ProductName;


Q9: CTE + window function, top 2 most expensive per category

In [ ]:
WITH ProductRank AS (
    SELECT
        c.CategoryName,
        p.ProductName,
        p.Price,
        ROW_NUMBER() OVER (
            PARTITION BY c.CategoryName
            ORDER BY p.Price DESC
        ) AS rn
    FROM Products p
    JOIN Categories c
        ON p.CategoryID = c.CategoryID
)
SELECT
    CategoryName,
    ProductName,
    Price
FROM ProductRank
WHERE rn <= 2
ORDER BY
    CategoryName,
    Price DESC;


Q10: Sakila – suggested queries
Assuming standard Sakila schema (customer, payment, rental, inventory, film, category, etc.).
​

In [ ]:
SELECT
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    c.email,
    SUM(p.amount) AS total_amount_spent
FROM customer c
JOIN payment p
    ON c.customer_id = p.customer_id
GROUP BY
    c.customer_id,
    customer_name,
    c.email
ORDER BY
    total_amount_spent DESC
LIMIT 5;


In [ ]:
SELECT
    cat.name AS category_name,
    COUNT(r.rental_id) AS rental_count
FROM category cat
JOIN film_category fc
    ON cat.category_id = fc.category_id
JOIN film f
    ON fc.film_id = f.film_id
JOIN inventory i
    ON f.film_id = i.film_id
JOIN rental r
    ON i.inventory_id = r.inventory_id
GROUP BY
    cat.category_id,
    cat.name
ORDER BY
    rental_count DESC
LIMIT 3;


In [ ]:
-- Films available per store
SELECT
    s.store_id,
    COUNT(DISTINCT i.film_id) AS films_available
FROM store s
JOIN inventory i
    ON s.store_id = i.store_id
GROUP BY
    s.store_id;

-- Films never rented per store
SELECT
    s.store_id,
    COUNT(DISTINCT i.film_id) AS films_never_rented
FROM store s
JOIN inventory i
    ON s.store_id = i.store_id
LEFT JOIN rental r
    ON i.inventory_id = r.inventory_id
WHERE r.rental_id IS NULL
GROUP BY
    s.store_id;


In [ ]:
SELECT
    YEAR(p.payment_date) AS year,
    MONTH(p.payment_date) AS month,
    SUM(p.amount) AS total_revenue
FROM payment p
WHERE YEAR(p.payment_date) = 2023
GROUP BY
    YEAR(p.payment_date),
    MONTH(p.payment_date)
ORDER BY
    year,
    month;


In [ ]:
SELECT
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    c.email,
    COUNT(r.rental_id) AS rental_count_last_6_months
FROM customer c
JOIN rental r
    ON c.customer_id = r.customer_id
WHERE r.rental_date >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH)
GROUP BY
    c.customer_id,
    customer_name,
    c.email
HAVING
    COUNT(r.rental_id) > 10
ORDER BY
    rental_count_last_6_months DESC;
